<a href="https://colab.research.google.com/github/victorsjc/data-science-labs/blob/main/labresults-analysis-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Instalar as bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re
import unicodedata
from typing import Dict, List, Tuple, Optional
import json

In [16]:
class LOINCMapper:
    """
    Algoritmo para mapeamento de exames clínicos e laboratoriais para códigos LOINC
    usando embeddings e técnicas híbridas de similaridade semântica.
    """

    def __init__(self, model_name: str = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
        """
        Inicializa o mapeador LOINC.

        Args:
            model_name: Nome do modelo de embeddings para usar
        """
        self.model = SentenceTransformer(model_name)
        self.vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)

        # Base de dados LOINC simplificada (em produção, usar base completa)
        self.loinc_database = self._create_sample_loinc_database()
        self.loinc_embeddings = None
        self.loinc_tfidf_matrix = None

        # Dicionário de sinônimos médicos
        self.medical_synonyms = self._create_medical_synonyms()

        self._prepare_loinc_embeddings()

    def _create_sample_loinc_database(self) -> pd.DataFrame:
        """Cria uma base de dados LOINC simplificada para demonstração."""
        sample_data = [
            {
                'loinc_code': '33747-0',
                'component': 'Hemoglobina',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Sangue',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Hemoglobina [Massa/volume] no Sangue',
                'synonyms': 'Hb, hemoglobina total, concentração de hemoglobina'
            },
            {
                'loinc_code': '789-8',
                'component': 'Eritrócitos',
                'property': 'NCnc',
                'time': 'Pt',
                'system': 'Sangue',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Eritrócitos [#/volume] no Sangue',
                'synonyms': 'RBC, hemácias, contagem de eritrócitos, glóbulos vermelhos'
            },
            {
                'loinc_code': '6690-2',
                'component': 'Leucócitos',
                'property': 'NCnc',
                'time': 'Pt',
                'system': 'Sangue',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Leucócitos [#/volume] no Sangue',
                'synonyms': 'WBC, glóbulos brancos, contagem de leucócitos'
            },
            {
                'loinc_code': '2093-3',
                'component': 'Colesterol',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Soro',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Colesterol [Massa/volume] no Soro ou Plasma',
                'synonyms': 'colesterol total, colesterolemia'
            },
            {
                'loinc_code': '33762-9',
                'component': 'Proteína C reativa',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Soro',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Proteína C reativa [Massa/volume] no Soro ou Plasma',
                'synonyms': 'PCR, CRP, proteína C-reativa'
            },
            {
                'loinc_code': '2160-0',
                'component': 'Creatinina',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Soro',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Creatinina [Massa/volume] no Soro ou Plasma',
                'synonyms': 'creatinina sérica'
            },
                        {
                'loinc_code': '33746-2',
                'component': 'Ácido ascórbico',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Soro',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Ácido ascórbico [Massa/volume] no Soro ou Plasma',
                'synonyms': 'vitamina C, ascorbato, ácido L-ascórbico, vitamina C sérica'
            },
            {
                'loinc_code': '5907-5',
                'component': 'Ácido ascórbico',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Plasma',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Ácido ascórbico [Massa/volume] no Plasma',
                'synonyms': 'vitamina C plasmática, ascorbato plasmático, concentração vitamina C'
            },
            {
                'loinc_code': '20567-4',
                'component': 'Ácido ascórbico',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Sangue total',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Ácido ascórbico [Massa/volume] no Sangue total',
                'synonyms': 'vitamina C total, ascorbato total, vitamina C no sangue'
            },
            {
                'loinc_code': '35668-3',
                'component': 'Ácido ascórbico',
                'property': 'MCnc',
                'time': 'Pt',
                'system': 'Urina',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Ácido ascórbico [Massa/volume] na Urina',
                'synonyms': 'vitamina C urinária, ascorbato urinário, excreção vitamina C'
            },
            {
                'loinc_code': '47132-6',
                'component': 'Ácido ascórbico',
                'property': 'MRto',
                'time': '24H',
                'system': 'Urina',
                'scale': 'Qn',
                'method': '',
                'display_name': 'Ácido ascórbico [Massa/tempo] na Urina de 24 horas',
                'synonyms': 'excreção de vitamina C 24h, clearance vitamina C, eliminação ascorbato'
            }
        ]
        return pd.DataFrame(sample_data)

    def _create_medical_synonyms(self) -> Dict[str, List[str]]:
        """Cria dicionário de sinônimos médicos."""
        return {
            'hemoglobina': ['hb', 'hemoglobina total', 'concentração hemoglobina'],
            'eritrocitos': ['rbc', 'hemacias', 'globulos vermelhos', 'contagem eritrocitos'],
            'leucocitos': ['wbc', 'globulos brancos', 'contagem leucocitos'],
            'colesterol': ['colesterol total', 'colesterolemia'],
            'proteina c reativa': ['pcr', 'crp', 'proteina c-reativa'],
            'creatinina': ['creatinina serica', 'creat'],
            'acido ascorbico': ['vitamina c', 'ascorbato', 'acido l-ascorbico', 'vit c'],
            'vitamina c': ['acido ascorbico', 'ascorbato', 'acido l-ascorbico', 'vit c'],
            'ascorbato': ['vitamina c', 'acido ascorbico', 'acido l-ascorbico'],
            'vitamina c serica': ['acido ascorbico serico', 'ascorbato serico'],
            'vitamina c plasmatica': ['acido ascorbico plasmatico', 'ascorbato plasmatico'],
            'vitamina c total': ['acido ascorbico total', 'ascorbato total'],
            'vitamina c urinaria': ['acido ascorbico urinario', 'ascorbato urinario'],
            'excrecao vitamina c': ['eliminacao vitamina c', 'clearance vitamina c']

        }

    def _normalize_text(self, text: str) -> str:
        """Normaliza texto removendo acentos e padronizando."""
        if not text:
            return ""

        # Remove acentos
        text = unicodedata.normalize('NFD', text)
        text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')

        # Converte para minúsculas e remove caracteres especiais
        text = re.sub(r'[^\w\s]', ' ', text.lower())
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    def _expand_synonyms(self, text: str) -> str:
        """Expande texto com sinônimos médicos."""
        normalized_text = self._normalize_text(text)
        words = normalized_text.split()

        expanded_terms = [normalized_text]

        for key, synonyms in self.medical_synonyms.items():
            if key in normalized_text:
                for synonym in synonyms:
                    expanded_terms.append(normalized_text.replace(key, synonym))

        return ' '.join(expanded_terms)

    def _prepare_loinc_embeddings(self):
        """Prepara embeddings e matriz TF-IDF para a base LOINC."""
        # Prepara textos expandidos para embedding
        loinc_texts = []
        for _, row in self.loinc_database.iterrows():
            combined_text = f"{row['display_name']} {row['component']} {row['synonyms']}"
            expanded_text = self._expand_synonyms(combined_text)
            loinc_texts.append(expanded_text)

        # Gera embeddings
        self.loinc_embeddings = self.model.encode(loinc_texts)

        # Gera matriz TF-IDF
        self.loinc_tfidf_matrix = self.vectorizer.fit_transform(loinc_texts)

    def _semantic_similarity(self, query_text: str) -> np.ndarray:
        """Calcula similaridade semântica usando embeddings."""
        expanded_query = self._expand_synonyms(query_text)
        query_embedding = self.model.encode([expanded_query])

        similarities = cosine_similarity(query_embedding, self.loinc_embeddings)[0]
        return similarities

    def _lexical_similarity(self, query_text: str) -> np.ndarray:
        """Calcula similaridade lexical usando TF-IDF."""
        expanded_query = self._expand_synonyms(query_text)
        query_tfidf = self.vectorizer.transform([expanded_query])

        similarities = cosine_similarity(query_tfidf, self.loinc_tfidf_matrix)[0]
        return similarities

    def map_to_loinc(self, exam_text: str, top_k: int = 3,
                     semantic_weight: float = 0.7,
                     lexical_weight: float = 0.3) -> List[Dict]:
        """
        Mapeia texto de exame para códigos LOINC.

        Args:
            exam_text: Texto do exame a ser mapeado
            top_k: Número de melhores correspondências a retornar
            semantic_weight: Peso da similaridade semântica
            lexical_weight: Peso da similaridade lexical

        Returns:
            Lista de dicionários com códigos LOINC e scores de similaridade
        """
        if not exam_text.strip():
            return []

        # Calcula similaridades
        semantic_scores = self._semantic_similarity(exam_text)
        lexical_scores = self._lexical_similarity(exam_text)

        # Combina scores (técnica híbrida)
        combined_scores = (semantic_weight * semantic_scores +
                          lexical_weight * lexical_scores)

        # Obtém os top_k resultados
        top_indices = np.argsort(combined_scores)[::-1][:top_k]

        results = []
        for idx in top_indices:
            loinc_row = self.loinc_database.iloc[idx]
            result = {
                'loinc_code': loinc_row['loinc_code'],
                'display_name': loinc_row['display_name'],
                'component': loinc_row['component'],
                'system': loinc_row['system'],
                'similarity_score': float(combined_scores[idx]),
                'semantic_score': float(semantic_scores[idx]),
                'lexical_score': float(lexical_scores[idx]),
                'confidence': self._calculate_confidence(combined_scores[idx])
            }
            results.append(result)

        return results

    def _calculate_confidence(self, score: float) -> str:
        """Calcula nível de confiança baseado no score."""
        if score >= 0.8:
            return 'Alta'
        elif score >= 0.6:
            return 'Média'
        elif score >= 0.4:
            return 'Baixa'
        else:
            return 'Muito Baixa'

    def batch_mapping(self, exam_texts: List[str]) -> Dict[str, List[Dict]]:
        """Mapeia múltiplos exames em lote."""
        results = {}
        for i, text in enumerate(exam_texts):
            results[f'exam_{i}'] = self.map_to_loinc(text)
        return results

    def evaluate_mapping(self, test_cases: List[Dict]) -> Dict:
        """
        Avalia performance do mapeamento com casos de teste.

        Args:
            test_cases: Lista de dicts com 'text' e 'expected_loinc'

        Returns:
            Métricas de avaliação
        """
        correct_predictions = 0
        total_cases = len(test_cases)

        for case in test_cases:
            predictions = self.map_to_loinc(case['text'], top_k=1)
            if predictions and predictions[0]['loinc_code'] == case['expected_loinc']:
                correct_predictions += 1

        accuracy = correct_predictions / total_cases if total_cases > 0 else 0

        return {
            'accuracy': accuracy,
            'correct_predictions': correct_predictions,
            'total_cases': total_cases
        }

In [17]:
# Exemplo de uso e demonstração
def demonstrate_loinc_mapping():
    """Demonstra o funcionamento do algoritmo com exemplos."""

    # Inicializa o mapeador
    mapper = LOINCMapper()

    # Exemplos de resultados de exames
    sample_exams = [
        "Hemoglobina: 12.5 g/dL",
        "Contagem de eritrócitos: 4.2 milhões/μL",
        "WBC: 7500/μL",
        "Colesterol total: 180 mg/dL",
        "PCR: 5.2 mg/L",
        "Creatinina sérica: 1.1 mg/dL",
        "Dosagem de hemoglobina no sangue",
        "Vitamina C: 31.9 mg/L"
    ]

    print("=== DEMONSTRAÇÃO DO ALGORITMO DE MAPEAMENTO LOINC ===\n")

    for i, exam in enumerate(sample_exams, 1):
        print(f"Exemplo {i}: {exam}")
        print("-" * 50)

        results = mapper.map_to_loinc(exam, top_k=2)

        if results:
            for j, result in enumerate(results, 1):
                print(f"  {j}º Melhor match:")
                print(f"    LOINC: {result['loinc_code']}")
                print(f"    Nome: {result['display_name']}")
                print(f"    Componente: {result['component']}")
                print(f"    Sistema: {result['system']}")
                print(f"    Score Total: {result['similarity_score']:.3f}")
                print(f"    Confiança: {result['confidence']}")
                print()
        else:
            print("  Nenhuma correspondência encontrada.\n")

        print("=" * 60 + "\n")

    # Demonstra avaliação com casos de teste
    print("=== AVALIAÇÃO DE PERFORMANCE ===\n")

    test_cases = [
        {'text': 'Hemoglobina no sangue', 'expected_loinc': '33747-0'},
        {'text': 'Contagem de glóbulos vermelhos', 'expected_loinc': '789-8'},
        {'text': 'Leucócitos totais', 'expected_loinc': '6690-2'}
    ]

    evaluation = mapper.evaluate_mapping(test_cases)
    print(f"Acurácia: {evaluation['accuracy']:.2%}")
    print(f"Casos corretos: {evaluation['correct_predictions']}/{evaluation['total_cases']}")


In [18]:
demonstrate_loinc_mapping()

=== DEMONSTRAÇÃO DO ALGORITMO DE MAPEAMENTO LOINC ===

Exemplo 1: Hemoglobina: 12.5 g/dL
--------------------------------------------------
  1º Melhor match:
    LOINC: 33747-0
    Nome: Hemoglobina [Massa/volume] no Sangue
    Componente: Hemoglobina
    Sistema: Sangue
    Score Total: 0.757
    Confiança: Média

  2º Melhor match:
    LOINC: 20567-4
    Nome: Ácido ascórbico [Massa/volume] no Sangue total
    Componente: Ácido ascórbico
    Sistema: Sangue total
    Score Total: 0.454
    Confiança: Baixa


Exemplo 2: Contagem de eritrócitos: 4.2 milhões/μL
--------------------------------------------------
  1º Melhor match:
    LOINC: 789-8
    Nome: Eritrócitos [#/volume] no Sangue
    Componente: Eritrócitos
    Sistema: Sangue
    Score Total: 0.675
    Confiança: Média

  2º Melhor match:
    LOINC: 6690-2
    Nome: Leucócitos [#/volume] no Sangue
    Componente: Leucócitos
    Sistema: Sangue
    Score Total: 0.524
    Confiança: Baixa


Exemplo 3: WBC: 7500/μL
-------------